In [17]:
import json
import pandas as pd


def read_data(path, limiter):
    """
    Args:
        path: path to dataset in jsonl file format
        limiter: number of rows

    Returns:
        data: json dict, where every line is one sample as json
    """
    with open(path) as file:
        data = [json.loads(next(file)) for x in range(limiter)]
    return data


def transform_dll_imports(json_sample):
    """
    Args:
        json_sample: one sample of dataset

    Returns:
        functions_dict: dict with all dll functions name with value True
    """
    imports = sample["imports"]
    functions_dict = {}
    for key in imports.keys():
        functions = imports[key]
        functions_with_values = {key.lower() + "-" + f_name: True for f_name in functions}
        functions_dict.update(functions_with_values)
    return functions_dict


def transform_dict(json_dict):
    """
    Args:
        json_dict: json dict with nested key-value, where value is list

    Returns:
        functions_dict: dict with value as a key and default value True
    """
    result_dict = {}
    for key in json_dict.keys():
        keys = json_dict[key]
        values_with_default = {key.lower() + "-" + f_name: True for f_name in keys}
        result_dict.update(values_with_default)
    return result_dict


def transform_list(json_list):
    """
    Args:
        json_list: json list of values

    Returns:
        functions_dict: dict with all dll functions name with value True
    """
    result_dict = {}
    result_dict.update({i: True for i in json_list})
    return result_dict


def flatten_json(y, separator=''):
    """
    Args:
        y: json object
        separator: separator

    Returns:
        functions_dict: dict with flatten values
    """
    out = {}

    def flatten(x, name=separator):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

In [18]:
# Extract "sha256", "md5", "appeared", "label" and "avclass" columns to a flat form

def get_simple_column(sample, columns = ["sha256", "md5", "appeared", "label", "avclass"]):
    '''
    input:
        data: variable with dataset
        columns: list of names extracted columns
    output:
        final_list: list of dicts, one list's element is a one sample of dataset
    '''
    simple_dict = {}
    simple_dict.update({column:sample[column] for column in columns})
    return simple_dict

In [19]:
# Extraction columns with list: "histogram", "byteentropy", "exports"

def get_simple_list_from_column(sample, columns = ["histogram", "byteentropy", "exports"]):

    '''
    input:
        data: variable with dataset
        columns: name of extracted columns (str)
    output:
        final_list: list of dicts, one list's element is a one sample of dataset
    '''
  
    dict_exports = {}
    dict_others = {}
    dict_final = {}

    for column in columns:
        if column == "exports":
            dict_exports.update({column+"_"+str(v).lower():True for v in sample[column]})
        else:
            dict_others.update({column+"_"+str(i):n for i, n in enumerate(sample[column])})             
                
    dict_final.update(dict_others)
    dict_final.update(dict_exports)
               
    return dict_final

In [20]:
# Extraction IMPORTS, GENERAL and STRINGS columns

def get_features_from_dict_column(sample, columns = "imports"):
    '''
    input:
            data
            columns
    output:
            functions_with_valuex
    '''
    dict_final = {}
    temp1 = {}
    temp2 = {}
    temp3 = {}

    feature = sample[columns]
    for key in feature.keys():
        content = feature[key]
        if (isinstance(content, list) and len(content)!=0):
            if not isinstance(content[0], str):
                temp1.update({columns +"_"+key.lower() + "-" + str(i):cont for i, cont in enumerate(content)})
            else:
                temp2.update({columns +"_"+ key.lower() + "-" + str(cont).lower():True for cont in content})
        else:
            temp3.update({columns+"_"+key:content})

    dict_final.update(temp1)   
    dict_final.update(temp2) 
    dict_final.update(temp3) 
    return dict_final

In [21]:
# Extraction HEADER column

def get_features_from_header(sample):
    '''
    input:
        data: data
    output:
        list_final: list of dicts
    '''    
    headers = sample["header"]
    dict_lists = {}
    dict_others = {}
    dict_final = {}

    for h in headers.keys():   
        temp = headers[h]      
        for k in temp.keys():  

            if isinstance(temp[k], list):
                dict_lists.update({"header_"+h.lower()+"_"+k.lower()+"_"+str(t).lower():True for t in temp[k]}) 
            else:
                dict_others.update({"header_"+h.lower()+"_"+k.lower():temp[k]})

    dict_final.update(dict_lists)    
    dict_final.update(dict_others)
    return dict_final

In [22]:
# Extraction Section column

def get_features_from_section(sample):

    '''
    input:
        data: data
    output:
        list_final: list of dicts
    '''  
    section_entry = sample['section']['entry']    # .text
    section_sections = sample['section']['sections']
    dict_lists_sections = {}
    dict_others_sections = {}
    dict_final = {}

    for part in section_sections:
        name = part['name']
        for position in ['size', 'entropy', 'vsize', 'props']:
            if position == 'props':
                dict_lists_sections.update({"section_"+section_entry.lower()+"_sections"+name.lower()+"_"+position+"_"+str(element).lower():True for element in part[position]})
            else:
                dict_others_sections.update({"section_"+section_entry.lower()+"_sections"+name.lower()+"_"+position:part[position]})
    dict_final.update(dict_lists_sections)
    dict_final.update(dict_others_sections)
    return dict_final

In [23]:
# Extraction DATADIRECTORIES column

def get_features_from_datadirectories(sample):
    '''
    input:
        data: data
    output:
        sum_others_datadir: list of dicts
    '''  

    dict_others_datadir = {}
    datadir = sample["datadirectories"]

    for element in datadir:
        element_name = element['name']
        for position in ['size', 'virtual_address']:
            dict_others_datadir.update({"datadirectories_"+element_name+"_"+position:element[position]})
    return dict_others_datadir

In [24]:
# 1. read datas
data_path = '/content/drive/MyDrive/Colab Notebooks/!Fellowship/Ember_dataset/sample_data_6k.jsonl'
data = read_data(data_path, limiter=)
# 2. iterate over dataset
flatten_dataset = []
for sample in data:
    # version 1 dummy way
    # flat = flatten_json(sample)
    # flatten_dataset.append(flat)
    # collect transformed values for each sample

    # version 2
    transformed = {}

    transformed.update(get_simple_column(sample))
    transformed.update(get_simple_list_from_column(sample))
    transformed.update(get_features_from_dict_column(sample, columns = "strings"))
    transformed.update(get_features_from_dict_column(sample, columns = "general"))
    transformed.update(get_features_from_dict_column(sample, columns = "imports"))
    transformed.update(get_features_from_header(sample))
    transformed.update(get_features_from_section(sample))
    transformed.update(get_features_from_datadirectories(sample))


    # here we fill with transform data
    flatten_dataset.append(transformed)

# change all transformed dataset to dataframe object
df = pd.DataFrame(flatten_dataset)
df

,sha256,md5,appeared,label,avclass,histogram_0,histogram_1,histogram_2,histogram_3,histogram_4,histogram_5,histogram_6,histogram_7,histogram_8,histogram_9,histogram_10,histogram_11,histogram_12,histogram_13,histogram_14,histogram_15,histogram_16,histogram_17,histogram_18,histogram_19,histogram_20,histogram_21,histogram_22,histogram_23,histogram_24,histogram_25,histogram_26,histogram_27,histogram_28,histogram_29,histogram_30,histogram_31,histogram_32,histogram_33,histogram_34,...,imports_userenv.dll-unregistergpnotification,imports_userenv.dll-ordinal130,imports_version.dll-getfileversioninfosizew,imports_version.dll-getfileversioninfow,imports_version.dll-verqueryvaluew,imports_winsta.dll-winstationrequestsessionslist,imports_winsta.dll-winstationquerylogoncredentialsw,imports_winsta.dll-winstationishelpassistantsession,imports_winsta.dll-winstationautoreconnect,imports_winsta.dll-_winstationwaitforconnect,imports_winsta.dll-winstationdisconnect,imports_winsta.dll-_winstationcallback,imports_winsta.dll-winstationnamefromlogonidw,imports_winsta.dll-_winstationfuscanremoteuserdisconnect,imports_winsta.dll-winstationenumerate_indexedw,imports_winsta.dll-winstationgetmachinepolicy,imports_winsta.dll-winstationqueryinformationw,imports_winsta.dll-winstationfreememory,imports_winsta.dll-winstationreset,imports_winsta.dll-_winstationnotifydisconnectpipe,imports_winsta.dll-winstationconnectw,imports_winsta.dll-winstationsetinformationw,imports_winsta.dll-winstationshutdownsystem,imports_winsta.dll-winstationcheckloopback,imports_winsta.dll-_winstationnotifylogon,imports_winsta.dll-_winstationnotifylogoff,imports_wintrust.dll-cryptcatcataloginfofromcontext,imports_wintrust.dll-cryptcatadmincalchashfromfilehandle,imports_wintrust.dll-cryptcatadminacquirecontext,imports_wintrust.dll-cryptcatadminenumcatalogfromhash,imports_wintrust.dll-cryptcatadminreleasecatalogcontext,imports_wintrust.dll-wthelperprovdatafromstatedata,imports_wintrust.dll-winverifytrust,imports_wintrust.dll-wthelpergetprovsignerfromchain,imports_wintrust.dll-cryptcatadminreleasecontext,imports_ws2_32.dll-ordinal116,imports_ws2_32.dll-getaddrinfo,imports_ws2_32.dll-ordinal115,header_coff_characteristics_aggressive_ws_trim,header_optional_dll_characteristics_terminal_server_aware
0,0abb4fda7d5b13801d63bee53e5e256be43e141faa077a...,63956d6417f8f43357d9a8e79e52257e,2006-12,0,,45521,13095,12167,12496,12429,11709,11864,12057,12881,11798,11802,11783,12029,12081,11756,12532,11980,11628,11504,11715,11809,12414,11779,11708,11956,11622,11859,11775,11717,11507,11873,11781,12015,11690,11676,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,c9cafff8a596ba8a80bafb4ba8ae6f2ef3329d95b85f15...,6f7bde7a1126debf0cc359a54953efc1,2007-01,0,,93059,15789,2871,3005,4107,1771,2918,4312,4610,903,1184,956,3099,1128,1639,2669,3139,1070,1723,1328,3020,3593,1224,975,2642,895,823,1147,1546,1871,674,747,3381,658,711,...,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
